In [ ]:
from hepmc.core.hamiltonian.spherical_hmc import StaticSphericalHMC
from hepmc.core.densities.gaussian import Gaussian
from hepmc.core.sampling import Sample
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib notebook

In [ ]:
D = 10
mu = np.full(D, 0.)
cov = np.empty((D, D))
for i in range(D):
    for j in range(D):
        cov[i, j] = 1/(1+abs(i-j))
target = Gaussian(D, mu, cov=cov)
lower = np.full(D, 0.)
upper = np.full(D, .5)
upper[0] = 5

In [ ]:
sample = target.rvs(10000000)

In [ ]:
sample = sample[np.where((sample > lower).all(axis=1))]
sample = sample[np.where((sample < upper).all(axis=1))]

In [ ]:
sample.shape

In [ ]:
sample_df = pd.DataFrame(sample)

In [ ]:
spherical_hmc.theta_to_x(np.full(D,0)+.5)

In [ ]:
spherical_hmc = StaticSphericalHMC(target, .08, .08, 4, 4, lim_lower=lower, lim_upper=upper)
start = D*[0.1]
spherical_hmc_sample = spherical_hmc.sample(20000, start)

In [ ]:
spherical_hmc_sample = Sample(data=spherical_hmc_sample.data[5000:], weights=spherical_hmc_sample.weights[5000:])

In [ ]:
spherical_hmc_df = pd.DataFrame(spherical_hmc_sample.data)

In [ ]:
spherical_hmc_sample.data.mean(axis=0)

In [ ]:
np.partition(spherical_hmc_sample.weights, -4)[-4:]

In [ ]:
np.argpartition(spherical_hmc_sample.weights, -4)[-4:]

In [ ]:
print('min:', spherical_hmc_sample.weights.min())
print('max:', spherical_hmc_sample.weights.max())
print('mean:', spherical_hmc_sample.weights.mean())
print('std:', spherical_hmc_sample.weights.std())

In [ ]:
#plt.figure(0)
#g = sns.PairGrid(sample_df)
#g = g.map_diag(sns.distplot, norm_hist=True, kde=False)
#g = g.map_offdiag(plt.hist2d, bins=20)
plt.figure(1)
h = sns.PairGrid(spherical_hmc_df)
h = h.map_diag(sns.distplot, norm_hist=True, kde=False, hist_kws={"weights" : spherical_hmc_sample.weights})
h = h.map_offdiag(plt.hist2d, weights=spherical_hmc_sample.weights, bins=20)
#h = h.map_diag(sns.distplot, norm_hist=True, kde=False)
#h = h.map_offdiag(plt.hist2d, bins=20)

In [ ]:
fig, ax = plt.subplots(1, 2, sharey='row', figsize=(10,5))
#ax1 = fig.add_subplot(1, 2, 1)
ax[0].plot(x, y, 'r-')
sns.distplot(spherical_hmc_df[0], ax=ax[0], kde=False, norm_hist=True, hist_kws={"weights" : spherical_hmc_sample.weights})
#ax2 = fig.add_subplot(1, 2, 2)
ax[1].plot(x, y, 'r-')
sns.distplot(spherical_hmc_df[1], ax=ax[1], kde=False, norm_hist=True, hist_kws={"weights" : spherical_hmc_sample.weights})

In [ ]:
util.effective_sample_size(spherical_hmc_sample, spherical_hmc_sample.mean, spherical_hmc_sample.variance)

In [ ]:
# resample
resamp_idx = np.random.choice(1000, 1000, p=spherical_hmc_sample.weights/spherical_hmc_sample.weights.sum())
spherical_hmc_resample = MarkovSample(data=spherical_hmc_sample.data[resamp_idx])
spherical_hmc_resample_df = pd.DataFrame(spherical_hmc_resample.data)

In [ ]:
fig, ax = plt.subplots(1, 2, sharey='row', figsize=(10,5))
#ax1 = fig.add_subplot(1, 2, 1)
ax[0].plot(x, y, 'r-')
sns.distplot(spherical_hmc_resample_df[0], ax=ax[0], kde=False, norm_hist=True)
#ax2 = fig.add_subplot(1, 2, 2)
ax[1].plot(x, y, 'r-')
sns.distplot(spherical_hmc_resample_df[1], ax=ax[1], kde=False, norm_hist=True)

In [ ]:
plt.figure()
grid = sns.jointplot(x=0, y=1, data=spherical_hmc_resample_df, kind="hex", marginal_kws={"norm_hist" : True})
grid.ax_marg_x.axes.plot(x, y, 'r-')
grid.ax_marg_y.axes.plot(y, x, 'r-')